[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/simple_csv_rag.ipynb)

# 用于CSV文件的简单RAG（检索增强生成）系统

## 概述

此代码实现了一个基本的检索增强生成（RAG）系统，用于处理和查询CSV文档。该系统将文档内容编码到向量存储中，然后可以查询该存储以检索相关信息。

# CSV文件结构和用例
CSV文件包含虚拟客户数据，包括名字、姓氏、公司等各种属性。该数据集将用于RAG用例，以方便创建客户信息问答系统。

## 核心组件

1. 加载和拆分csv文件。
2. 使用[FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)和OpenAI嵌入创建向量存储
3. 设置检索器以查询已处理的文档
4. 基于csv数据创建问答功能。

## 方法详情

### 文档预处理

1. 使用langchain Csvloader加载csv文件
2. 将数据拆分为块。


### 向量存储创建

1. 使用OpenAI嵌入创建文本块的向量表示。
2. 从这些嵌入创建FAISS向量存储，以实现高效的相似性搜索。

### 检索器设置

1. 配置检索器以获取给定查询最相关的块。

## 此方法的优势

1. 可扩展性：通过分块处理可以处理大型文档。
2. 灵活性：易于调整块大小和检索结果数量等参数。
3. 效率：利用FAISS在高维空间中进行快速相似性搜索。
4. 与先进NLP集成：使用OpenAI嵌入进行最先进的文本表示。

## 结论

这个简单的RAG系统为构建更复杂的信息检索和问答系统提供了坚实的基础。通过将文档内容编码到可搜索的向量存储中，它能够高效地检索相关信息以响应查询。这种方法对于需要快速访问csv文件中特定信息的应用程序特别有用。

导入库

# 软件包安装和导入

下面的单元格安装了运行此笔记本所需的所有必要软件包。


In [ ]:
# Install required packages
!pip install faiss-cpu langchain langchain-community langchain-openai pandas python-dotenv

In [8]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# CSV文件结构和用例
CSV文件包含虚拟客户数据，包括名字、姓氏、公司等各种属性。该数据集将用于RAG用例，以方便创建客户信息问答系统。

In [ ]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/customers-100.csv https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv


In [18]:
import pandas as pd

file_path = ('data/customers-100.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


load and process csv data

In [19]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

Initiate faiss vector store and openai embedding

In [9]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query(" ")))
vector_store = FAISS(
    embedding_function=OpenAIEmbeddings(),
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

Add the splitted csv data to the vector store

In [ ]:
vector_store.add_documents(documents=docs)

Create the retrieval chain

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Query the rag bot with a question based on the CSV data

In [15]:
answer= rag_chain.invoke({"input": "which company does sheryl Baxter work for?"})
answer['answer']

'Sheryl Baxter works for Rasmussen Group.'